In [1]:
import pandas as pd, geopandas as gpd, numpy as np, matplotlib.pyplot as plt, seaborn as sns, topojson as tp
import requests, json, time
from shapely.ops import cascaded_union
sns.set(style="whitegrid", font_scale=1.25)

geodata = gpd.read_file("blocks.geojson")

residential = ["RZ1: SUBURBAN", "RZ2: SUBURBAN CORE", "RZ3: URBAN RESIDENTIAL", "RZ4: MEDIUM DENSITY RESIDENTIAL", "RZ5: HIGH DENSITY RESIDENTIAL"]

def sort_zones(policy):
    if ("PRZ2: " in policy):
        return np.nan
    elif policy in residential:
        return residential.index(policy)
    elif ("CZ5: " in policy) | ("CZ2: " in policy):
        return 5
    elif policy in "DES: DESIGNATED":
        return 6
    else:
        return None

data = geodata[["DIVISION_NAME", "DISTRICT_NAME", "BLOCK_DERIVED_AREA", "LAND_USE_POLICY_ZONES", "OVERLAY_PROVISION_ZONES", "CURRENT_LIFECYCLE_STAGE", "geometry"]]
data.columns = ["division", "district", "block", "zone", "overlay", "current", "geometry"]
data = data[data.overlay.apply(lambda x: ("FUA: " not in x) if x != None else True)][["division", "district", "block", "zone", "current", "geometry"]]
data = data[data["current"] == "REGISTERED"][["district", "block", "zone", "geometry"]]
data.loc[:, "block"] = data.block.astype("int")
data.loc[:, "zone"] = data.zone.apply(sort_zones)
data.dropna(inplace=True)
data.loc[:, "zone"] = data.zone.astype("int")
data.loc[:, "central"] = data.district.apply(lambda x: 1 if x == "CANBERRA CENTRAL" else 0)
# data = data[["central", "block", "zone", "geometry"]]
data.to_file("housing.geojson", driver="GeoJSON")

In [2]:
central_0 = gpd.GeoDataFrame(columns=["zone", "geometry"])

for zone in data[data.central == 1].zone.unique():
    polygons = data[(data.central == 1) & (data.zone == zone)].geometry
    central_0.loc[len(central_0)] = [zone, cascaded_union(polygons)]

zone_labels = ["RZ1", "RZ2", "RZ3", "RZ4", "RZ5", "other", "DES"]
def label_zones(zone):
    return zone_labels[zone]

central_0.loc[:, "zone"] = pd.to_numeric(central_0["zone"])

central_0.to_file("central_0.geojson", driver="GeoJSON")
# central_0.sort_values("zone", inplace=True)
# central_0.loc[:, "zone"] = central_0.zone.apply(label_zones)
# central_0.plot("zone", cmap="viridis_r", figsize=(30, 15), linewidth=0, legend=True).set_axis_off()

data1 = data.copy()
data2 = data.copy()

for i in data1.index:
    if (data1.at[i, "central"] == 1) & (data1.at[i, "block"] > 799) & (data1.at[i, "zone"] < 2):        
        data1.at[i, "zone"] = 2

for i in data2.index:
    if (data2.at[i, "central"] == 1) & (data2.at[i, "block"] > 699) & (data2.at[i, "zone"] < 2):        
        data2.at[i, "zone"] = 2
        
central_1 = gpd.GeoDataFrame(columns=["zone", "geometry"])
central_2 = gpd.GeoDataFrame(columns=["zone", "geometry"])

for zone in data1[data1.central == 1].zone.unique():
    polygons = data1[(data1.central == 1) & (data1.zone == zone)].geometry
    central_1.loc[len(central_1)] = [zone, cascaded_union(polygons)]

central_1.loc[:, "zone"] = pd.to_numeric(central_1["zone"])
central_1.to_file("central_1.geojson", driver="GeoJSON")
# central_1.sort_values("zone", inplace=True)
# central_1.loc[:, "zone"] = central_1.zone.apply(label_zones)
# central_1.plot("zone", cmap="viridis_r", figsize=(30, 15), linewidth=0, legend=True).set_axis_off()

for zone in data2[data2.central == 1].zone.unique():
    polygons = data2[(data2.central == 1) & (data2.zone == zone)].geometry
    central_2.loc[len(central_2)] = [zone, cascaded_union(polygons)]

central_2.loc[:, "zone"] = pd.to_numeric(central_2["zone"])
central_2.to_file("central_2.geojson", driver="GeoJSON")
# central_2.sort_values("zone", inplace=True)
# central_2.loc[:, "zone"] = central_2.zone.apply(label_zones)
# central_2.plot("zone", cmap="viridis_r", figsize=(30, 15), linewidth=0, legend=True).set_axis_off()

In [3]:
topo = tp.Topology(
    central_0,
    toposimplify=1e-4,
    prevent_oversimplify=True
).topoquantize(1e5)
topo.to_json("central_0.topojson")

topo = tp.Topology(
    central_1,
    toposimplify=1e-4,
    prevent_oversimplify=True
).topoquantize(1e5)
topo.to_json("central_1.topojson")

topo = tp.Topology(
    central_2,
    toposimplify=1e-4,
    prevent_oversimplify=True
).topoquantize(1e5)
topo.to_json("central_2.topojson")

In [4]:
act_0 = gpd.GeoDataFrame(columns=["zone", "geometry"])

for zone in data.zone.unique():
    polygons = data[data["zone"] == zone]["geometry"]
    act_0.loc[len(act_0)] = [zone, cascaded_union(polygons)]

act_0.loc[:, "zone"] = pd.to_numeric(act_0["zone"])
act_0.to_file("act_0.geojson", driver="GeoJSON")
# act_0.sort_values("zone", inplace=True)
# act_0.loc[:, "zone"] = act_0.zone.apply(label_zones)
# act_0.plot("zone", cmap="viridis_r", figsize=(30, 15), linewidth=0, legend=True).set_axis_off()

In [5]:
topo = tp.Topology(
    act_0,
    toposimplify=1e-4,
    prevent_oversimplify=True
).topoquantize(1e5)
topo.to_json("act_0.topojson")

In [21]:
test = data[data["central"] == 1]
test[(test["zone"] < 2) & (test["block"] > 699)]

,district,block,zone,geometry,central
8,CANBERRA CENTRAL,743,0,"POLYGON ((149.14826 -35.26029, 149.14799 -35.2...",1
9,CANBERRA CENTRAL,780,0,"POLYGON ((149.14777 -35.25999, 149.14781 -35.2...",1
10,CANBERRA CENTRAL,1017,0,"POLYGON ((149.14781 -35.25969, 149.14785 -35.2...",1
11,CANBERRA CENTRAL,743,0,"POLYGON ((149.14812 -35.25947, 149.14839 -35.2...",1
12,CANBERRA CENTRAL,969,0,"POLYGON ((149.14839 -35.25950, 149.14862 -35.2...",1
...,...,...,...,...,...
146449,CANBERRA CENTRAL,2729,1,"POLYGON ((149.10856 -35.31411, 149.10851 -35.3...",1
146647,CANBERRA CENTRAL,3262,1,"POLYGON ((149.15212 -35.33215, 149.15208 -35.3...",1
147049,CANBERRA CENTRAL,1521,0,"POLYGON ((149.13209 -35.33011, 149.13228 -35.3...",1
147118,CANBERRA CENTRAL,1848,1,"POLYGON ((149.15380 -35.33440, 149.15373 -35.3...",1


In [7]:
data[(data.zone < 2) & (data.block < 4000)].groupby("district").median().sort_values("block")

,block,zone,central
district,,,
MAJURA,214.0,0.0,0.0
GUNGAHLIN,480.0,0.0,0.0
MOLONGLO VALLEY,517.0,0.0,0.0
BELCONNEN,791.0,0.0,0.0
CANBERRA CENTRAL,796.0,0.0,1.0
WESTON CREEK,811.0,0.0,0.0
TUGGERANONG,826.0,0.0,0.0
WODEN VALLEY,863.0,0.0,0.0
JERRABOMBERRA,1012.0,0.0,0.0


In [9]:
796/517


1.539651837524178